In [1]:
#
# David Laziuk
# Naive Bayes algorithm from scratch
#

In [2]:
#Imports
import math
import numpy as np
import pandas as pd

In [3]:
data=pd.read_csv('breast-cancer-wisconsin.csv',na_values= "?")
#Dropping rows with NAs
data=data.dropna()
#Dropping Sample ID Column
data=data.iloc[:,1:]
print("Original Shape: "+str(data.shape))
data.head()

Original Shape: (683, 10)


,F1,F2,F3,F4,F5,F6,F7,F8,F9,Class
0,5,1,1,1,2,1.0,3,1,1,2
1,5,4,4,5,7,10.0,3,2,1,2
2,3,1,1,1,2,2.0,3,1,1,2
3,6,8,8,1,3,4.0,3,7,1,2
4,4,1,1,3,2,1.0,3,1,1,2


In [4]:
#Data is already in a normal range

In [5]:
#Shuffling Entries
data=data.sample(frac=1).reset_index(drop=True)
#70/30 split into train/test data
train_size=int(np.floor(data.shape[0]*.7))
train=data.iloc[0:train_size,:]
test=data.iloc[train_size:,:]
del train_size

In [6]:
#Seperating Training Data
g=train.groupby(train.Class)
train2=g.get_group(2)
train4=g.get_group(4)
#Prior Probabilities
prior2=train2.shape[0]/train.shape[0]
prior4=train4.shape[0]/train.shape[0]

c2size=train2.shape[0]
c4size=train4.shape[0]
#Class Conditionals Probs
#Class=2
cc2=np.zeros((10,9))
#Populating 10x9 matrix with conditional probabilities for each val 1-10 for cols F1-F9
for i in range(9):
    colname="F"+str(i+1)
    cc=train2[colname].value_counts()
    cc=cc/c2size
    for j in range(cc.shape[0]):
        cc2[int(cc.index[j]-1),i]=cc.iloc[j]
#Class=4
cc4=np.zeros((10,9))
#Populating 10x9 matrix with conditional probabilities for each val 1-10 for cols F1-F9
for i in range(9):
    colname="F"+str(i+1)
    cc=train4[colname].value_counts()
    cc=cc/c4size
    for j in range(cc.shape[0]):
        cc4[int(cc.index[j]-1),i]=cc.iloc[j]
del g,c2size,c4size,i,j,colname,cc

In [7]:
#Defining Function to make prediction for 1 new testing sample
def makePred(testRow):
    #Start with prior probability
    p2=prior2
    p4=prior4
    #Looping through F1->F9
    for i in range(testRow.shape[0]-1):
        colname="F"+str(i+1)
        #Extract New Value from Column
        val=testRow[colname]
        #Multiply prediction by conditional probability of new value
        p2*=cc2[int(val-1),i]
        p4*=cc4[int(val-1),i]
    #Extract actual class
    actual=testRow.iloc[9]
    if(p2>p4):
        return 2,actual
    else:
        return 4,actual

In [8]:
#Testing
count=0
for i in range(test.shape[0]):
    pred,truth=makePred(test.iloc[i,:])
    if(pred==truth):
        count+=1
print(str(round((count/test.shape[0])*100,2))+"% accuracy")
del i,pred,truth,count

98.05% accuracy
